In [ ]:
%reload_ext autoreload
%autoreload 2

# Wikipedia

In [1]:
from coleta.corretor_paginas_wikipedia import CorretorEstruturaCategorias
from coleta.wikipedia import CategoriaCollector, PaginaCollector, WikipediaDownloader
from urllib.parse import urlencode
from pymongo import MongoClient
import time
import pandas as pd

wikipedia_downloader = WikipediaDownloader()
pagina_collector = PaginaCollector(collection_paginas='pages_brasil', collection_paginas_conteudo='pages_content_brasil')
categoria_collector = CategoriaCollector(collection_categorias='categories_brasil', pagina_collector=pagina_collector)
country = 'Brasil'

## Definir categoria raiz

Em nosso caso de estudo a categoria raiz sera Geografia do Brasil. Abaixo esta o documento correspondente, obtido atraves de procura feita 
no resultado da consulta de category_members da categoria "Categoria:Brasil".

In [ ]:
is_adicionar_categoria_raiz = False
if is_adicionar_categoria_raiz:
    categoria_geografia_brasil = {'pageid': 15122,
    'ns': 14,
    'title': 'Categoria:Geografia do Brasil',
    'type': 'subcat', 
    'country': 'Brasil'}
    categoria_collector.add_categories_for_check([categoria_geografia_brasil])
# category_members = wikipedia_downloader.get_category_members(cmtitle='Categoria:Brasil')

## Iteração de download de categorias

- Categories status
    * Check: waiting for someone to check if it has to be downloaded
    * Waiting: waiting for dowload
    * Skeep: skeep download of this category
    * Done: downloaded

### Passo 1 - Procurar categorias candidatas a download

In [ ]:
categorias_candidatas_para_download = categoria_collector.get_categories_for_check()

In [ ]:
for ccpd in categorias_candidatas_para_download[:10]:
    print(f'{ccpd["pageid"]} - {ccpd["title"]}')

### Passo 2 - Selecionar para download e skeep

In [ ]:
categorias_selecionadas_para_download = []
if len(categorias_selecionadas_para_download) > 0:
    categoria_collector.set_categories_for_download(categorias_selecionadas_para_download)

In [ ]:
categorias_selecionadas_para_skeep = []
if len(categorias_selecionadas_para_skeep) > 0:
    categoria_collector.set_categories_for_skeep(categorias_selecionadas_para_skeep)

### Passo 3 - Fazer download de selecionadas

In [ ]:
categorias_para_download = categoria_collector.get_categories_for_download()
print(f'{len(categorias_para_download)} categorias recuperadas para download...')
for cpd in categorias_para_download:
    categoria_collector.download_category_tree(cpd, cpd['country'])

## Corrigir estrutura de categorias

In [ ]:
# corretor_estrutura_categorias = CorretorEstruturaCategorias(collection_name_incorreto='', collection_correto='')

## Download de páginas (wikitext)

- Pages status (wikitext)
    * Waiting: waiting for download
    * Done: Download done

In [ ]:
# Executar apenas uma vez
# pagina_collector.set_pages_for_wikitext_download()

In [ ]:
paginas_download_wikitext = pagina_collector.get_pages_for_download()

In [ ]:
pagina_collector.run_wikitext_download(5)

## Download de páginas (text)

- Pages status (text)
    * Waiting: waiting for download
    * Donw: Download done

In [ ]:
# Executar apenas uma vez
# pagina_collector.set_pages_for_text_download()

In [ ]:
keep_running = True
while keep_running:
    amount_done = pagina_collector.run_text_download(5)
    keep_running = amount_done > 0

## Backup do que já foi feito download

In [ ]:
run_backup = False
if run_backup:
    categories = list(mongo_categories.find())
    categories_df = pd.DataFrame(categories)
    categories_df.to_csv('categories.csv')

    pages = list(mongo_pages.find())
    pages_df = pd.DataFrame(pages)
    pages_df.to_csv('pages.csv')

### Verificar categorias e páginas sem country

In [ ]:
# Pages
pages_no_country = list(db.pages.find({"country": {"$exists": False }}))
print(f'{len(pages_no_country)} pages found with no country')

# Categories
categories_no_country = list(db.categories.find({"country": {"$exists": False }}))
print(f'{len(categories_no_country)} categories found with no country')

In [ ]:
def show_download_statistics():
    pdc = get_pages_downloads_counts()
    print(f'Page download counts: {pdc}')
    pages_wikitext_counts = mongo_content.count_documents({})
    print(f'Pages with wikitext: {pages_wikitext_counts}')
# show_download_statistics()

### Download page texts

In [ ]:
run_text_download(303)
show_text_download_statistics()

In [ ]:
pcs = list(mongo_content.aggregate(
    [ 
        { '$group': { '_id': '$pageid', 'total': { '$sum': 1 } } }
    ]
))

In [ ]:
dobrado = [p for p in pcs if p['total'] > 1]


In [ ]:
def find_distinct_pages():
    return list(mongo_pages.aggregate(
        [ 
            { '$group': { '_id': '$pageid', 'total': { '$sum': 1 } } },
            { '$sort': { 'total': -1 }}
        ]
    ))

def count_distinct_pages():
    return len(find_distinct_pages())

print(f'We have downloaded {count_distinct_pages()} distinct pages')

In [ ]:
list(mongo_pages.find({ 'pageid': 158914 }))

In [ ]:
pages_dobrado = [p for p in pcs if p['total'] > 1]
pages_dobrado[0]

In [ ]:
list(mongo_pages.find({'pageid': 237330}))

## Correção collection pages

- Problema: a mesma página tinha sido baixada mais de uma vez porque era referenciada por várias coleções
- Solução
    * Renomear collection pages para pages_incorreto (feito no mongo shell)
    * Recriar a collection pages com estrutura correta
        * A categoria agora será um objeto com id e título ao invés de estes valores estarem soltos na páginas. Assim ficará melhor estuturado
        * O objeto da página terá uma collection de categorias. Cada página lida da estrutura antiga terá na nova collection um array com as categorias que a referenciam
    * Renomear pages_content para pages_content incorreto

In [ ]:
from pymongo import MongoClient

mongo_client = MongoClient()
db = mongo_client.wikipedia
collection_incorreto = db['pages_brasil_incorreto']
collection_correto = db['pages_brasil']

In [ ]:
# Obtem todas paginas de collection incorreta
todas_paginas_incorreto = list(collection_incorreto.find({}))

In [ ]:
def criar_objetos_categoria(todas_paginas_incorreto):
    # Corrigir categoria das paginas incorretas. Cria objeto para categoria e coloca id e titulo
    for pagina_incorreto in todas_paginas_incorreto:
        category = {
            'category_id': pagina_incorreto['category_id'],
            'category_title': pagina_incorreto['category_title']
        }
        pagina_incorreto['category'] = category
        del pagina_incorreto['category_id']
        del pagina_incorreto['category_title']
    return todas_paginas_incorreto

todas_paginas_incorreto = criar_objetos_categoria(todas_paginas_incorreto)

In [ ]:
def agregar_categorias_por_pagina(todas_paginas_incorreto):
    # Dictionario que mapeia uma pageid para documento da pagina
    agregado_paginas_incorreto = {}

    for pagina_incorreto in todas_paginas_incorreto:
        pageid_incorreto = pagina_incorreto['pageid']
        if pageid_incorreto in agregado_paginas_incorreto:
            existente = agregado_paginas_incorreto[pageid_incorreto]
            existente['categories'].append(pagina_incorreto['category'])
        else:
            pagina_incorreto['categories'] = [pagina_incorreto['category']]
            agregado_paginas_incorreto[pageid_incorreto] = pagina_incorreto
        del pagina_incorreto['category']
    
    return agregado_paginas_incorreto

agregado_paginas_incorreto = agregar_categorias_por_pagina(todas_paginas_incorreto)

In [ ]:
# Insere as páginas corrigidas no mongo
paginas_agregadas = list(agregado_paginas_incorreto.values())
collection_correto.insert_many(paginas_agregadas)

In [ ]:
# Conta páginas para ver o resultado
print(collection_correto.count_documents({}))
print(collection_incorreto.count_documents({}))

## Análise documentos download

In [ ]:
import seaborn as sns
import pandas as pd

### Análise wikitext

In [ ]:
pcs = list(mongo_content.find({}))

In [ ]:
lengths_wikitext = [len(pc['wikitext']) for pc in pcs]
df = pd.DataFrame(lengths_wikitext, columns=['Length'])
df.head()

In [ ]:
sns.displot(df, x='Length', bins=20)

In [ ]:
# Verificando se todas as páginas srealmente têm conteúdo
df['Length'].describe()

## Fix downloaded text

- It was stored in the documents the whole structure of the answer
- It should store only the ext

In [ ]:
pcs = list(mongo_content.find({}))
for pc in pcs:
    mongo_content.update_one(
        { 'pageid': pc['pageid'] },
        { '$set': { 'text': pc['text']['parse']['text'] } }
    )

### Análise text

In [ ]:
pcs = list(mongo_content.find({}))

In [ ]:
lengths_text = [len(pc['text']) for pc in pcs]
df = pd.DataFrame(lengths_text, columns=['Length'])
df.head()

In [ ]:
sns.displot(df, x='Length', bins=20)

In [ ]:
# Verificando se todas as páginas srealmente têm conteúdo
df['Length'].describe()